In [28]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 2
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="6-31g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

mycc = cc.CCSD(mf)
mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Thu Oct 23 16:25:28 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000

init E= -0.735581811516521
  alpha nocc = 1  HOMO = -0.428875905563132  LUMO = 0.0906074258757933
  beta  nocc = 1  HOMO = -0.422782006847908  LUMO = 0.0917604853842934
cycle= 1 E= -1.08366735877223  delta_E= -0.348  |g|= 0.0132  |ddm|= 0.44
  alpha nocc = 1  HOMO = -0.516473146105886  LUMO = 0.150366241490128
  beta  nocc = 1  HOMO = -0.516748549197009  LUMO = 0.15033165994403
cycle= 2 E= -1.08380719741209  delta_E= -0.00014  |g|= 0.00207  |ddm|= 0.0129
  alpha nocc = 1  HOMO = -0.514626233136186  LUMO = 0.152275422459565
  beta  nocc = 1  HOMO = -0.514705981622363  LUMO = 0.152265686992383
cycle= 3 E= -1.08381071641424  delta_E= -3.52e-06  |g|= 7.74e-05  |ddm|= 0.00239
  alpha nocc = 1  HOMO = -0.514661268399487  LUMO = 0.152269818323371
  beta  nocc = 1  HOMO = -0.5146618299203  LUMO = 0.152269730439392
cycle= 4 E= -1.0838107234146  delta_E= -7e-09  |g|= 1.18e-07  |ddm|= 0.000128
  alpha nocc = 1  HOMO = -0.514663233143063  LUMO = 0.152269737626045
  beta  nocc = 1  HOMO = -0.514663

-0.03400559715686858

In [12]:
def thouless_trans(t1):
    ''' thouless transformation |psi'> = exp(t1)|psi>
        gives the transformed mo_occrep in the 
        original mo basis <psi_p|psi'_i>
        t = t_ia
        t_ia = c_ik c.T_ka
        c_ik = <psi_i|psi'_k>
    '''
    q, r = jnp.linalg.qr(t1,mode='complete')
    u_ji = q
    u_ai = r.T
    u_occ = jnp.vstack((u_ji,u_ai))
    mo_t, _ = jnp.linalg.qr(u_occ,mode='complete')
    return mo_t

In [29]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'uhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
# mycc.t1 = t1
# ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
pyscf_interface.prep_afqmc(mycc,options,chol_cut=1e-6)

from ad_afqmc import config, mpi_jax, wavefunctions
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

import h5py
chol_file='FCIDUMP_chol'
with h5py.File(chol_file, "r") as fh5:
    [nelec, nmo, ms, nchol] = fh5["header"]
    h0 = jnp.array(fh5.get("energy_core"))
    h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
    h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)


#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (1, 1)
# Number of basis functions: 4
# Number of Cholesky vectors: 10
#
# Number of MPI ranks: 1
#
# norb: 4
# nelec: (1, 1)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 3
# seed: 2
# walker_type: rhf
# trial: uhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [30]:
ham_data['h1_mod'] = h1_mod
mo_coeff = jnp.array(np.load('mo_coeff.npz')["mo_coeff"])
wave_data['mo_B'] = mo_coeff[1]

t2aa = mycc.t2[0] # + 2 * np.einsum("ia,jb->ijab", mycc.t1[0], mycc.t1[0])
t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
t2aa = t2aa.transpose(0, 2, 1, 3)
t2bb = mycc.t2[2] # + 2 * np.einsum("ia,jb->ijab", mycc.t1[1], mycc.t1[1])
t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
t2bb = t2bb.transpose(0, 2, 1, 3)
t2ab = mycc.t2[1] # + np.einsum("ia,jb->ijab", mycc.t1[0], mycc.t1[1])
t2ab = t2ab.transpose(0, 2, 1, 3)
t1a = np.array(mycc.t1[0])
t1b = np.array(mycc.t1[1])

mo_ta = thouless_trans(t1a)
mo_tb = thouless_trans(t1b)
noccA, noccB = trial.nelec[0], trial.nelec[1]
wave_data['mo_ta'] = mo_ta[:,:noccA]
wave_data['mo_tb'] = mo_tb[:,:noccB]
mo_a_A = wave_data['mo_ta'] # @ wave_data['mo_coeff'][0]
mo_b_B = wave_data['mo_tb'] # @ wave_data["mo_B"].T @ wave_data['mo_coeff'][1]

wave_data["rot_t1A"] = mo_a_A[:noccA,:noccA].T @ t1a
wave_data["rot_t2AA"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_a_A[:noccA,:noccA].T,mo_a_A[:noccA,:noccA].T,t2aa)
wave_data["rot_t1B"] = mo_b_B[:noccB,:noccB].T @ t1b
wave_data["rot_t2BB"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_b_B[:noccB,:noccB].T,mo_b_B[:noccB,:noccB].T,t2bb)
wave_data["rot_t2AB"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_a_A[:noccA,:noccA].T,mo_b_B[:noccB,:noccB].T,t2ab)

In [31]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1


ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
print(mf.e_tot - prop_data['e_estimate'])
print('cc energy: ', mycc.e_tot)
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
# print(trial._calc_overlap(wave_data['mo_ta'],wave_data["mo_B"]@wave_data['mo_tb'],wave_data))
print(trial._calc_energy(wave_data['mo_ta'],wave_data["mo_B"]@wave_data['mo_tb'],ham_data,wave_data))

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -1.0838107234146097
mf enegry:  -1.083810723414609
6.661338147750939e-16
cc energy:  -1.1178163205714777
-1.083790174018175


In [32]:
wave_data['mo_coeff']

[Array([[1.00000000e+00],
        [7.60686089e-17],
        [1.38777878e-16],
        [1.82801028e-16]], dtype=float64),
 Array([[1.00000000e+00],
        [4.02789862e-16],
        [1.45395063e-09],
        [1.96548302e-16]], dtype=float64)]

In [ ]:
energy_samples = jnp.real(
            trial.calc_energy(prop_data["walkers"], ham_data, wave_data)
        )

In [43]:
trial.calc_energy(prop_data["walkers"], ham_data, wave_data)

Array([-1.08381072+0.j, -1.08381072+0.j, -1.08381072+0.j], dtype=complex128)

In [44]:
type(prop_data["walkers"])

jaxlib.xla_extension.ArrayImpl

In [46]:
trial

uhf(norb=4, nelec=(1, 1), n_opt_iter=30, n_batch=1)

In [45]:
trial._calc_energy_restricted(prop_data["walkers"][0], ham_data, wave_data)

Array(-1.08381072+0.j, dtype=complex128)

In [60]:
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print(mf.e_tot+eccsd)

-1.589947461477418


In [69]:
@partial(jit, static_argnums=0)
def _tls_olp(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    wave_data: dict,
) -> complex:
    '''<exp(T1)HF|walker>'''

    # walker_dn = wave_data['mo_B'].T @ walker_dn
    olp = jnp.linalg.det(
        wave_data["mo_ta"].T.conj() @ walker_up
    ) * jnp.linalg.det(wave_data["mo_tb"].T.conj() @ walker_dn)

    return olp

@partial(jit, static_argnums=5)
def _tls_exp1(x: float, h1_mod: jax.Array, walker_up: jax.Array,
                    walker_dn: jax.Array, wave_data: dict, trial):
    '''
    unrestricted <ep(T1)HF|exp(x*h1_mod)|walker>/<HF|walker>
    '''
    walker_up_1x = walker_up + x * h1_mod.dot(walker_up)
    walker_dn_1x = walker_dn + x * h1_mod.dot(walker_dn)
    walker_dn_1x = wave_data['mo_B'].T @ walker_dn_1x
    olp = _tls_olp(trial,walker_up_1x, walker_dn_1x, wave_data)
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)

    return olp/o0

@partial(jit, static_argnums=5)
def _tls_exp2(x: float, chol_i: jax.Array, walker_up: jax.Array,
                   walker_dn: jax.Array, wave_data: dict, trial) -> complex:
    '''
    <exp(T1)HF|exp(x*h2_mod)|walker>/<HF|walker>
    '''

    walker_up_2x = (
        walker_up
        + x * chol_i.dot(walker_up)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_up))
    )
    walker_dn_2x = (
        walker_dn
        + x * chol_i.dot(walker_dn)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_dn))
    )
    walker_dn_2x = wave_data['mo_B'].T @ walker_dn_2x
    olp = _tls_olp(trial, walker_up_2x,walker_dn_2x,wave_data)
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)
    
    return olp/o0

In [70]:
from jax import lax
@partial(jit, static_argnums=0)
def _ut2_walker_olp(
    trial, walker_up: jax.Array, walker_dn: jax.Array, wave_data: dict
) -> complex:
    '''<exp(T1)HF|(t1+t2)|walker> = (t_ia G_ia + t_iajb G_iajb) * <exp(T1)HF|walker>'''
    noccA, t2AA = trial.nelec[0], wave_data["rot_t2AA"]
    noccB, t2BB = trial.nelec[1], wave_data["rot_t2BB"]
    t2AB = wave_data["rot_t2AB"]
    # convert trial_mo_dn into beta mo_basis
    # wave_data["mo_B"] = <psi_a|psi_b>
    mo_A = wave_data['mo_ta'] #@ wave_data["mo_coeff"][0]
    mo_B = wave_data['mo_tb'] #@ wave_data["mo_B"].T @ wave_data["mo_coeff"][1]
    green_a = (
        walker_up.dot(jnp.linalg.inv(mo_A.T.conj() @ walker_up))
    ).T
    # convert walker_dn into beta mo_basis
    # s.t. green_b in beta basis thus can be contracted with beta amplitude
    walker_dn = wave_data["mo_B"].T @ walker_dn
    green_b = (
        walker_dn.dot(jnp.linalg.inv(mo_B.T.conj() @ walker_dn))
    ).T
    green_a, green_b = green_a[:noccA, noccA:], green_b[:noccB, noccB:]
    o0 = _tls_olp(trial,walker_up,walker_dn,wave_data)
    # o1 = jnp.einsum("ia,ia", t1A, green_a) + jnp.einsum("ia,ia", t1B, green_b)
    o2 = (
        0.5 * jnp.einsum("iajb, ia, jb", t2AA, green_a, green_a)
        + 0.5 * jnp.einsum("iajb, ia, jb", t2BB, green_b, green_b)
        + jnp.einsum("iajb, ia, jb", t2AB, green_a, green_b)
    )
    return o2 * o0

@partial(jit, static_argnums=5)
def _ut2_exp1(x: float, h1_mod: jax.Array, walker_up: jax.Array,
                    walker_dn: jax.Array, wave_data: dict, trial):
    '''
    unrestricted <ep(T1)HF|T2 exp(x*h1_mod)|walker>/<HF|walker>
    '''
    walker_up_1x = walker_up + x * h1_mod.dot(walker_up)
    walker_dn_1x = walker_dn + x * h1_mod.dot(walker_dn)
    
    olp = _ut2_walker_olp(trial,walker_up_1x, walker_dn_1x, wave_data)
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)

    return olp/o0

@partial(jit, static_argnums=5)
def _ut2_exp2(x: float, chol_i: jax.Array, walker_up: jax.Array,
                   walker_dn: jax.Array, wave_data: dict, trial) -> complex:
    '''
    t_ia <psi_i^a|exp(x*h2_mod)|walker>/<HF|walker>
    '''

    walker_up_2x = (
        walker_up
        + x * chol_i.dot(walker_up)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_up))
    )
    walker_dn_2x = (
        walker_dn
        + x * chol_i.dot(walker_dn)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_dn))
    )
    
    olp = _ut2_walker_olp(trial, walker_up_2x,walker_dn_2x,wave_data)
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)
    
    return olp/o0

@partial(jit, static_argnums=0)
def _calc_energy_pt2(trial, walker_up, walker_dn, ham_data, wave_data):
    '''
    t1 = <exp(T1)HF|walker>/<HF|walker>
    t2 = <exp(T1)HF|T1+T2|walker>/<HF|walker>
    e0 = <exp(T1)HF|h1+h2|walker>/<HF|walker>
    e1 = <exp(T1)HF|(T1+T2)(h1+h2)|walker>/<HF|walker>
    '''

    eps=1e-4

    norb = trial.norb
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # e0 = <exp(T1)HF|h1+h2|walker>/<HF|walker> #
    # one body
    x = 0.0
    f1 = lambda a: _tls_exp1(a,h1_mod,walker_up,walker_dn,wave_data,trial)
    t1, d_exp1 = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        eps,walker_up,walker_dn,wave_data = carry
        return carry, _tls_exp2(eps,c,walker_up,walker_dn,wave_data,trial)

    _, exp2_p = lax.scan(scanned_fun, (eps,walker_up,walker_dn,wave_data), chol)
    _, exp2_0 = lax.scan(scanned_fun, (0.0,walker_up,walker_dn,wave_data), chol)
    _, exp2_m = lax.scan(scanned_fun, (-1.0 * eps,walker_up,walker_dn,wave_data), chol)
    d2_exp2 = (exp2_p - 2.0 * exp2_0 + exp2_m) / eps / eps

    e0 = (d_exp1 + jnp.sum(d2_exp2) / 2.0 )

    d_exp1 = d2_exp2 = None
    exp2_p = exp2_0 = exp2_m = None
    
    # e1 = <exp(T1)HF|(T1+T2)(h1+h2)|walker>/<HF|walker>
    # one body
    x = 0.0
    f1 = lambda a: _ut2_exp1(a,h1_mod,walker_up,walker_dn,wave_data,trial)
    t2, d_exp1 = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        eps,walker_up,walker_dn,wave_data = carry
        return carry, _ut2_exp2(eps,c,walker_up,walker_dn,wave_data,trial)

    _, exp2_p = lax.scan(scanned_fun, (eps,walker_up,walker_dn,wave_data), chol)
    _, exp2_0 = lax.scan(scanned_fun, (0.0,walker_up,walker_dn,wave_data), chol)
    _, exp2_m = lax.scan(scanned_fun, (-1.0 * eps,walker_up,walker_dn,wave_data), chol)
    d2_exp2 = (exp2_p - 2.0 * exp2_0 + exp2_m) / eps / eps

    e1 = (d_exp1 + jnp.sum(d2_exp2) / 2.0 )

    return jnp.real(t1), jnp.real(t2), jnp.real(e0), jnp.real(e1)

In [57]:
print(mf.e_tot,mycc.e_tot)

-1.58975539667538 -1.620397631502197


In [ ]:
# h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0

In [71]:
t1,t2,e0,e1 = _calc_energy_pt2(trial, walker_up, walker_dn, ham_data, wave_data)
print(t1,t2,e0,e1)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)

-0.9976266447650609 -7.612534329717163e-34 2.8332074418015942 0.030377900489296384
-1.6203978159737895
